# CISC 867 Project

## Metadata


### Authors

- Dominic Phillips
- Bennet Montgomery

### Important links

- [Original paper](https://paperswithcode.com/paper/unsupervised-any-to-many-audiovisual)
  - [Code](https://github.com/dunbar12138/Audiovisual-Synthesis)
- [Google Drive folder](https://drive.google.com/drive/folders/12O-IlQd4ZM2kxPVJ9hitSUfuSrprPIEC)
- [Project proposal](https://queensuca-my.sharepoint.com/personal/16drp4_queensu_ca/_layouts/15/Doc.aspx?sourcedoc=%7B04585478-D0CA-47C6-B276-3098828A7DAB%7D&file=Project%20Proposal.pptx&action=edit&mobileredirect=true)
- [OnQ deliverables](https://onq.queensu.ca/d2l/le/content/582452/Home)
  - [Project draft](https://onq.queensu.ca/d2l/lms/dropbox/user/folder_submit_files.d2l?db=258790&grpid=593240&isprv=0&bp=0&ou=582452)
    - [Rubric](https://onq.queensu.ca/d2l/lp/rubrics/preview.d2l?ou=582452&rubricId=47677&originTool=quicklinks)
    - [Draft meeting spreadsheet](https://docs.google.com/spreadsheets/d/1ZefaU8afq16ykSjEAJRLccg8th2RtwL6R4Jt-lg6nqM/edit#gid=0)
    - [Overleaf project](https://www.overleaf.com/project/6171e082af75ef1ea42efe1f)
- [Tensorflow tutorial](https://www.tensorflow.org/tutorials/audio/simple_audio#spectrogram)

### Background links

- [What is audio](https://youtu.be/m3XbqfIij_Y)

### What success will look like

- Reproduce the objective acoustic analysis (VCTK) to evaluate the model performance. We should arrive within 0.5 of the SCA value calculated by the paper for their model and 2 for the MCD.​
- Training a model faster than the 3 minutes the paper claims​
- Alternatively, failing to reproduce the objective analysis claimed in the paper and showing the research to be irreproducible. ​

## Workflow

### Clone our repo

In [1]:
!git clone https://github.com/CISC-867/Project

Cloning into 'Project'...
remote: Enumerating objects: 576, done.
remote: Counting objects: 100% (576/576), done.
remote: Compressing objects: 100% (379/379), done.
remote: Total 576 (delta 255), reused 443 (delta 122), pack-reused 0
Receiving objects: 100% (576/576), 195.18 MiB | 30.95 MiB/s, done.
Resolving deltas: 100% (255/255), done.
Checking out files: 100% (125/125), done.


### Setup VSCode server

https://stackoverflow.com/a/59537334/11141271  
https://stackoverflow.com/questions/51882170/connect-to-google-collab-with-ssh-from-console-from-pc


In [ ]:
import random, string
password = ''.join(random.choice(string.ascii_letters + string.digits) for i in range(20))

#Download ngrok
! wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip -qq -n ngrok-stable-linux-amd64.zip
#Setup sshd
! apt-get install -qq -o=Dpkg::Use-Pty=0 openssh-server pwgen > /dev/null
#Set root password
! echo root:$password | chpasswd
! mkdir -p /var/run/sshd
! echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
! echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config
! echo "LD_LIBRARY_PATH=/usr/lib64-nvidia" >> /root/.bashrc
! echo "export LD_LIBRARY_PATH" >> /root/.bashrc

#Run sshd
get_ipython().system_raw('/usr/sbin/sshd -D &')

#Ask token
print("Copy authtoken from https://dashboard.ngrok.com/auth")
import getpass
authtoken = getpass.getpass()



Creating config file /etc/ssh/sshd_config with new version
Creating SSH2 RSA key; this may take some time ...
2048 SHA256:23bEm3KYBm+Y+Ptu/B1NRjpOXp7arRjuewBE6bAz1s0 root@7c0498d6a560 (RSA)
Creating SSH2 ECDSA key; this may take some time ...
256 SHA256:Rm62dn3Qu+kEx6IPZG2Ayc3K0/IilNTzXtGfQt2VRto root@7c0498d6a560 (ECDSA)
Creating SSH2 ED25519 key; this may take some time ...
256 SHA256:iB55wQSLXT81u0Trnf5nqCFHSJ9nm+4eTWZYMTRXHhA root@7c0498d6a560 (ED25519)
Created symlink /etc/systemd/system/sshd.service → /lib/systemd/system/ssh.service.
Created symlink /etc/systemd/system/multi-user.target.wants/ssh.service → /lib/systemd/system/ssh.service.
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Copy authtoken from https://dashboard.ngrok.com/auth
··········


In [ ]:
#Create tunnel
get_ipython().system_raw('./ngrok authtoken $authtoken && ./ngrok tcp 22 &')
#Print root password
print("Root password: {}".format(password))



Root password: u16xBpqROlJBKoLrsg5C


In [ ]:
#Get public address
! sleep 2 && curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"
# might need to run twice

tcp://6.tcp.ngrok.io:10886


Use main.ipynb in vscode from here out.

In [ ]:
print("idle")

idle


## Clone WaveRNN

In [22]:
!pip install webrtcvad

     |████████████████████████████████| 66 kB 2.5 MB/s 
  Created wheel for webrtcvad: filename=webrtcvad-2.0.10-cp37-cp37m-linux_x86_64.whl size=72410 sha256=c61e57515664bc257657c6abf57bbc3dea84da4fefbe3dc79c28ec6146257492
  Stored in directory: /root/.cache/pip/wheels/11/f9/67/a3158d131f57e1c0a7d8d966a707d4a2fb27567a4fe47723ad
Successfully built webrtcvad


In [3]:
%cd Project/

/content/Project


In [17]:
!mkdir checkpoints


mkdir: cannot create directory ‘checkpoints’: File exists


In [18]:
!mkdir checkpoints/tom

mkdir: cannot create directory ‘checkpoints/tom’: File exists


In [19]:
!cd checkpoints/tom && wget "https://cisc867.blob.core.windows.net/checkpoints/tom/model306255.pth"

--2021-12-05 18:00:50--  https://cisc867.blob.core.windows.net/checkpoints/tom/model306255.pth
Resolving cisc867.blob.core.windows.net (cisc867.blob.core.windows.net)... 20.60.220.36
Connecting to cisc867.blob.core.windows.net (cisc867.blob.core.windows.net)|20.60.220.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 294997144 (281M) [application/octet-stream]
Saving to: ‘model306255.pth’

model306255.pth     100%[===================>] 281.33M  26.8MB/s    in 9.6s    

2021-12-05 18:01:00 (29.2 MB/s) - ‘model306255.pth’ saved [294997144/294997144]



In [20]:
!cd checkpoints/tom && wget "https://cisc867.blob.core.windows.net/checkpoints/tom/model306255_vocoder.pth"

--2021-12-05 18:01:18--  https://cisc867.blob.core.windows.net/checkpoints/tom/model306255_vocoder.pth
Resolving cisc867.blob.core.windows.net (cisc867.blob.core.windows.net)... 20.60.220.36
Connecting to cisc867.blob.core.windows.net (cisc867.blob.core.windows.net)|20.60.220.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17995109 (17M) [application/octet-stream]
Saving to: ‘model306255_vocoder.pth’

model306255_vocoder 100%[===================>]  17.16M  31.4MB/s    in 0.5s    

2021-12-05 18:01:19 (31.4 MB/s) - ‘model306255_vocoder.pth’ saved [17995109/17995109]



In [2]:
# Clone the repo
# https://github.com/as-ideas/ForwardTacotron
# https://colab.research.google.com/github/as-ideas/ForwardTacotron/blob/master/notebooks/synthesize.ipynb#scrollTo=LHfPLGkqe-Jx
!git clone https://github.com/as-ideas/ForwardTacotron
# Checkout to latest version as of the time this notebook was created
!cd ForwardTacotron && git reset --hard "f537c5f16f9459948b64f563749d65dcedacbe04"

Cloning into 'ForwardTacotron'...
remote: Enumerating objects: 5473, done.
remote: Counting objects: 100% (1844/1844), done.
remote: Compressing objects: 100% (744/744), done.
remote: Total 5473 (delta 1342), reused 1512 (delta 1095), pack-reused 3629
Receiving objects: 100% (5473/5473), 205.55 MiB | 30.01 MiB/s, done.
Resolving deltas: 100% (3752/3752), done.
HEAD is now at f537c5f Update README.md


In [7]:
import importlib
import model.trainer
import torch
import data.VCTK

In [8]:
importlib.reload(data.VCTK)
dataset = data.VCTK.VCTKDataset(
    text_file_paths=["resources/tomscott/txt/tomscott.txt"],
    audio_file_paths=["resources/tomscott/wav48/tomscott.wav"]
)
print(f"Loaded {len(dataset)} entries")

Loaded 1 entries


In [23]:
importlib.reload(model.trainer)
trainer = model.trainer.Trainer(
    device=torch.device("cuda"),
    checkpoint=306255,
    checkpoint_dir="checkpoints/tom",
    load_from_checkpoint=True,
)

Using default model
Using default vocoder
Loading models from checkpoint 306255
Models loaded


In [25]:
trainer.train(
    epochs=10,
    dataset=dataset,
    save_every_n=10,
    batch_size=10,
    run_name="runs/tom3"
)

Beginning training 10 epochs, logging every 1, saving every 10, batch size 10.


RuntimeError: ignored